In [1]:
import pandas as pd
sia_df=pd.read_parquet('datasets/twitter/sia_vct.parquet') #FT and transformer
users_df=pd.read_csv('datasets/twitter/user_pos_neg_ids_27_3.csv')
dblp_df=pd.read_parquet('datasets/dblpv13_fasttext.parquet', engine='pyarrow')

In [2]:
from ast import literal_eval

In [3]:
literal_eval(users_df['outIDs'][0])[0]

'S2589152922000485'

In [24]:
user_v_ft, user_v_tr=[],[]
br=0
for i, row in users_df.iterrows():
    uv_ft=0.
    uv_tr=0.
    num_has=0
    arts=literal_eval(row.outIDs)
    for a in arts:
        try:
            lookup=sia_df.loc[sia_df['artID']==a].values[0]
            uv_ft+=lookup[4] #zbrajamo pojedinačne vektore svakog članka
            uv_tr+=lookup[3]
            num_has+=1

        except:# neki članci nisu mogli biti dohvaćeni putem APIja
            br+=1

        

    if type(uv_ft)==type(0.) or num_has < 2: #nema vektoriziranih članaka 
        user_v_ft.append(None)
        user_v_tr.append(None)
    else:   
        uv_ft=uv_ft/num_has #dijelimo brojem članaka
        uv_tr=uv_tr/num_has
        user_v_ft.append(uv_ft)
        user_v_tr.append(uv_tr)


In [25]:
user_v=pd.DataFrame({
    'userID': list(users_df['userID']),
    'ftvect': user_v_ft,
    'trvect': user_v_tr
})

In [26]:
user_v=user_v.dropna() #makni one koji nisu imali vektorizirane članke ili su imali samo jedan

In [27]:
print("sveukupno imamo", len(user_v), "korisnika")

sveukupno imamo 395 korisnika


In [28]:
user_v

,userID,ftvect,trvect
0,4910170583,"[-0.015486389, 0.0041903406, 0.014377301, 0.05...","[-0.015645882, -0.009310105, -0.018257707, 0.0..."
1,973267080781942785,"[-0.0023664555, 0.013145153, -0.0040409225, 0....","[-0.043928728, -0.085610054, 0.007853937, -0.0..."
2,1188667249025925120,"[-0.015840998, 0.016070688, 0.005398368, 0.053...","[-0.019852461, 0.03437543, -0.007818024, 0.010..."
3,1143872062336983040,"[-0.00792171, 0.007946789, 0.0044410536, 0.052...","[-0.04176928, 0.019971866, -0.07198615, 0.0037..."
4,805048451880939520,"[-0.010990788, -2.3457455e-05, 0.010952565, 0....","[0.009853591, -0.019459577, 0.0059503503, 0.03..."
...,...,...,...
407,935335706418753537,"[-0.018632151, -0.007448089, -0.01047132, 0.05...","[0.056930788, -0.062114056, 0.01893661, 0.0193..."
408,2786148233,"[0.0046785497, 0.013557263, -0.0008513846, 0.0...","[0.0050746673, 0.023642873, -0.009931956, -0.0..."
409,1247872005912891392,"[-0.017615106, 0.0008072568, 0.0082156705, 0.0...","[-0.019035894, -0.0596074, 0.018317144, 0.0089..."
410,1467005139454095363,"[0.004418953, 0.00562046, 0.0045859497, 0.0558...","[-0.0063024415, -0.048351463, -0.0063766376, 0..."


In [29]:
user_v.to_parquet('datasets/tw_0604_user_vectors.parquet', engine='pyarrow')